In [2]:
import os
import pandas as pd

In [4]:
data_path = '../COVID-19/csse_covid_19_data/csse_covid_19_time_series/'
confirm_df = pd.read_csv(
    os.path.join(data_path,'time_series_19-covid-Confirmed.csv')
).drop(columns=['Lat', 'Long']).rename(columns={"Province/State": "state", "Country/Region": "country"})
death_df = pd.read_csv(
    os.path.join(data_path,'time_series_19-covid-Deaths.csv')
).drop(columns=['Lat', 'Long']).rename(columns={"Province/State": "state", "Country/Region": "country"})
recovery_df = pd.read_csv(
    os.path.join(data_path,'time_series_19-covid-Recovered.csv')
).drop(columns=['Lat', 'Long']).rename(columns={"Province/State": "state", "Country/Region": "country"})

confirm_df.sample(5)

,state,country,1/22/20,1/23/20,1/24/20,1/25/20,1/26/20,1/27/20,1/28/20,1/29/20,...,2/25/20,2/26/20,2/27/20,2/28/20,2/29/20,3/1/20,3/2/20,3/3/20,3/4/20,3/5/20
91,NaN,Austria,0,0,0,0,0,0,0,0,...,2,2,3,3,9,14,18,21,29,41
74,NaN,Iran,0,0,0,0,0,0,0,0,...,95,139,245,388,593,978,1501,2336,2922,3513
159,"Contra Costa County, CA",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1
171,"Santa Rosa County, FL",US,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1
155,NaN,Gibraltar,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,1,1


In [17]:
c_df = confirm_df.melt(id_vars=['country', 'state'], var_name="date", value_name="confirmed")
d_df = death_df.melt(id_vars=['country', 'state'], var_name="date", value_name="dead")
r_df = recovery_df.melt(id_vars=['country', 'state'], var_name="date", value_name="recovered")
df = c_df.merge(d_df, on=['country', 'state', 'date']).merge(r_df, on=['country', 'state', 'date'])
df.head()

,country,state,date,confirmed,dead,recovered
0,Mainland China,Anhui,1/22/20,1,0,0
1,Mainland China,Beijing,1/22/20,14,0,0
2,Mainland China,Chongqing,1/22/20,6,0,0
3,Mainland China,Fujian,1/22/20,1,0,0
4,Mainland China,Gansu,1/22/20,0,0,0


In [18]:
df[['date', 'dead']].values.tolist()[:5]

[['1/22/20', 0],
 ['1/22/20', 0],
 ['1/22/20', 0],
 ['1/22/20', 0],
 ['1/22/20', 0]]

To make a gapminder-like plot, we want to make a list of locations. 

`[ {"Country": "Mainland China", ... 
    "deaths": [["1/22/20", 0], ... }`

In [19]:
data = []
for group in df.groupby(['country']):
    temp_df = group[1].groupby(['country', 'date'])[['confirmed', 'dead', 'recovered']].sum()
    country_data = {"country": group[0], 
                    "confirmed": temp_df[["date", "confirmed"]].values.tolist(),
                    "dead": group[1][["date", "dead"]].values.tolist(),
                    "recovered": group[1][["date", "recovered"]].values.tolist(),
                   }
    data.append(country_data)

In [20]:
import json
open('cntry_stat.json', 'w').write(json.dumps(data))

334098

In [39]:
from pprint import pprint
for group in df.groupby(['country']):
    if group[0] == "Mainland China":
        temp_df = group[1].groupby('date')[['confirmed', 'dead', 'recovered']].sum().reset_index()
        print(temp_df[['date', 'confirmed']].values.tolist())
        #pprint(list(zip(temp_df['confirmed'].index.values.tolist(), temp_df['confirmed'].values.tolist())))

[['1/22/20', 547], ['1/23/20', 639], ['1/24/20', 916], ['1/25/20', 1399], ['1/26/20', 2062], ['1/27/20', 2863], ['1/28/20', 5494], ['1/29/20', 6070], ['1/30/20', 8124], ['1/31/20', 9783], ['2/1/20', 11871], ['2/10/20', 42306], ['2/11/20', 44327], ['2/12/20', 44699], ['2/13/20', 59832], ['2/14/20', 66292], ['2/15/20', 68347], ['2/16/20', 70446], ['2/17/20', 72364], ['2/18/20', 74139], ['2/19/20', 74546], ['2/2/20', 16607], ['2/20/20', 74999], ['2/21/20', 75472], ['2/22/20', 76922], ['2/23/20', 76938], ['2/24/20', 77152], ['2/25/20', 77660], ['2/26/20', 78065], ['2/27/20', 78498], ['2/28/20', 78824], ['2/29/20', 79251], ['2/3/20', 19693], ['2/4/20', 23680], ['2/5/20', 27409], ['2/6/20', 30553], ['2/7/20', 34075], ['2/8/20', 36778], ['2/9/20', 39790], ['3/1/20', 79826], ['3/2/20', 80026], ['3/3/20', 80151], ['3/4/20', 80271]]


In [46]:
'/'.join([x.zfill(2) for x in '1/22/20'.split('/')])

'01/22/20'

In [41]:
'1'.zfill(2)

'01'

In [48]:
import datetime

In [53]:
month, day, year = map(int, '1/22/20'.split('/'))
datetime.date(year + 2000, month, day).isoformat()

'2020-01-22'

In [58]:
def convert_date(x):
    month, day, year = map(int, x.split('/'))
    year = 2000 + year
    return datetime.date(year, month, day).isoformat()

df.date = df.date.apply(convert_date)

In [5]:
df.head()

NameError: name 'df' is not defined

In [11]:
confirm_df = pd.read_csv(
    os.path.join(data_path,'time_series_19-covid-Confirmed.csv')
).rename(columns={"Province/State": "state", "Country/Region": "country", "Lat": "lat", "Long": "lng"})[['country', 'state', 'lat', 'lng']]

confirm_df.to_csv('coordinates.csv', index=False)

,country,state,lat,lng
0,Mainland China,Anhui,31.82570,117.2264
1,Mainland China,Beijing,40.18240,116.4142
2,Mainland China,Chongqing,30.05720,107.8740
3,Mainland China,Fujian,26.07890,117.9874
4,Mainland China,Gansu,36.06110,103.8343
5,Mainland China,Guangdong,23.34170,113.4244
6,Mainland China,Guangxi,23.82980,108.7881
7,Mainland China,Guizhou,26.81540,106.8748
8,Mainland China,Hainan,19.19590,109.7453
9,Mainland China,Hebei,38.04280,114.5149


# OWID data


In [13]:
import pandas as pd

In [14]:
df = pd.read_csv('../data_sources/our_world_in_data/owid_ts.csv', index_col=0, parse_dates=True).rename(
    columns={'location': 'country_name'}).fillna(0, downcast='infer')
df.head()

,country_name,new_cases,new_deaths,total_cases,total_deaths
date,,,,,
2020-02-25,Afghanistan,0,0,1,0
2020-02-26,Afghanistan,0,0,1,0
2020-02-27,Afghanistan,0,0,1,0
2020-02-28,Afghanistan,0,0,1,0
2020-02-29,Afghanistan,0,0,1,0


In [29]:
name2code = pd.read_csv('../output/metadata/country/country_name_code.csv', index_col="country_name").to_dict()['country_code']

In [30]:
df.loc[:, "country_code"] = df.country_name.apply(lambda x: name2code[x])

In [68]:
idx = pd.date_range(df.index.min(), df.index.max())

for group in df.groupby("country_code"):
    if group[0] == 'GBR':
        x = group[1][['country_name', 'total_cases']].reindex(idx, fill_value=0)
        print(x)

              country_name  total_cases
2020-01-21               0            0
2020-01-22               0            0
2020-01-23               0            0
2020-01-24               0            0
2020-01-25               0            0
2020-01-26               0            0
2020-01-27               0            0
2020-01-28               0            0
2020-01-29               0            0
2020-01-30               0            0
2020-01-31               0            0
2020-02-01  United Kingdom            2
2020-02-02  United Kingdom            2
2020-02-03  United Kingdom            2
2020-02-04  United Kingdom            2
2020-02-05  United Kingdom            2
2020-02-06  United Kingdom            2
2020-02-07  United Kingdom            3
2020-02-08  United Kingdom            3
2020-02-09  United Kingdom            3
2020-02-10  United Kingdom            4
2020-02-11  United Kingdom            8
2020-02-12  United Kingdom            8
2020-02-13  United Kingdom            9
